# [自然言語処理100本ノック 第2章: UNIXコマンドの基礎](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch2)

[hightemp.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt)は，日本の最高気温の記録を「都道府県」「地点」「℃」「日」のタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，[hightemp.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt)を入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．

ということで, [hightemp.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt)をダウンロード

In [22]:
import urllib.request
urllib.request.urlretrieve("http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt",
                           "hightemp.txt")

('hightemp.txt', <http.client.HTTPMessage at 0x7fb15596e320>)

まぁ curl, wgetでもできるけど, pythonで統一したかった. 一応下に書いとく.

In [ ]:
!curl -O http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt

In [ ]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/hightemp.txt

## 10. 行数のカウント
行数をカウントせよ．確認にはwcコマンドを用いよ．

In [58]:
def test_chap2_10():
    filename = "hightemp.txt"
    answer = !grep '' {filename}|wc -l
    answer = int(answer[0])
    assert line_count(filename) == answer
    
    print("\"{}\"の行数は{}行です".format(filename, answer))

確認のwcコマンドだが, [このサイト](http://blog.unfindable.net/archives/6937)によると

> 「wc -l file」が数えるのは行数ではなく改行数なので、ファイルの末尾が改行でない場合、行数よりも1少ない結果が返ります

ということなので, まぁ今回の場合はwcそのまま使ってでも大丈夫なんだけど, 今回は  
```
grep '' file|wc -l
```
を使う

In [68]:
def line_count(filename):
    num_lines = sum(1 for line in open(filename))
    return num_lines

In [69]:
test_chap2_10()

"hightemp.txt"の行数は24行です


sum(1 for line in open(filename)) は早いらしい. 他の実装も色々こちらに書いてある

https://stackoverflow.com/questions/845058/how-to-get-line-count-cheaply-in-python

ちなみに, 展開するとこんな感じのコード

```
new_list = []
for _ in open(filename)
    new_list.append(1)
    
sum(new_list)
```

リストの部分は省略されているので, 付け加えてもいい

sum(\[1 for line in open(filename)\])

In [65]:
[1 for line in range(10)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## 11. タブをスペースに置換
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [123]:
def test_chap2_11():
    filename = "hightemp.txt"
    
    sed = !sed "s/$(printf "\t")/ /g" {filename}
    tr = !cat {filename} | tr "\t" " "
    expand = !expand -t 1 {filename}
    
    assert sed == tr
    assert tr == expand
    assert replace_tab_to_space(filename).rstrip("\n") == sed.n.rstrip("\n")
    
    print(replace_tab_to_space(filename))

[こちら](https://rcmdnk.com/blog/2016/09/13/computer-gnu-bsd-linux-mac/)によると, sedはLinux（GNU）とMac（BSD）ではタブの指定方法が違うらしいので, 単純に\tと打ってもダメみたい. (ちなみに, このノートブックの実行環境はLinuxです

なので, 両方共使える 
``` 
sed "s/$(printf "\t")/ /g" 
```
とわざわざしている

In [124]:
def replace_tab_to_space(filename):
    with open(filename) as f:
        text = f.read().expandtabs(1)
    return text

In [125]:
test_chap2_11()

高知県 江川崎 41 2013-08-12
埼玉県 熊谷 40.9 2007-08-16
岐阜県 多治見 40.9 2007-08-16
山形県 山形 40.8 1933-07-25
山梨県 甲府 40.7 2013-08-10
和歌山県 かつらぎ 40.6 1994-08-08
静岡県 天竜 40.6 1994-08-04
山梨県 勝沼 40.5 2013-08-10
埼玉県 越谷 40.4 2007-08-16
群馬県 館林 40.3 2007-08-16
群馬県 上里見 40.3 1998-07-04
愛知県 愛西 40.3 1994-08-05
千葉県 牛久 40.2 2004-07-20
静岡県 佐久間 40.2 2001-07-24
愛媛県 宇和島 40.2 1927-07-22
山形県 酒田 40.1 1978-08-03
岐阜県 美濃 40 2007-08-16
群馬県 前橋 40 2001-07-24
千葉県 茂原 39.9 2013-08-11
埼玉県 鳩山 39.9 1997-07-05
大阪府 豊中 39.9 1994-08-08
山梨県 大月 39.9 1990-07-19
山形県 鶴岡 39.9 1978-08-03
愛知県 名古屋 39.9 1942-08-02



ちょっとずるしました.

```
assert replace_tab_to_space(filename).rstrip("\n") == sed.n.rstrip("\n")
```

なんですが, sedは IPython.utils.text.SList になっており, それに .n をつけると, 改行コードでリストをつなげてくれるらしいです. しかし, それだと末尾に改行コードはつかないです. (わかる方は, "\n".join(sed)を想像するといいと思います.

replace_tab_to_spaceメソッドは, そのまま文字列のタブを置換するので, 最後の改行が残り, assert error となります. なのでズルして, 末尾の改行コードは取るようにしましたw

## 12. 1列目をcol1.txtに，2列目をcol2.txtに保存
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [145]:
def test_chap2_12():
    filename = "hightemp.txt"
    
    list_col1 = get_col_list(filename, col=0, delimiter="\t")
    list_col2 = get_col_list(filename, col=1, delimiter="\t")
    
    cut_col1 = !cut -f 1 {filename}
    cut_col2 = !cut -f 2 {filename}
    
    assert list_col1 == cut_col1
    assert list_col2 == cut_col2 
    
    assert save_list_to_file(list_col1, "col1.txt", delimiter="\n") == True
    assert save_list_to_file(list_col2, "col2.txt", delimiter="\n") == True
    
    print(list_col1)
    print(list_col2)

In [154]:
def get_col_list(filename, col=0, delimiter=" "):
    list_col = []
    with open(filename) as f:
        for line in f:
            col_word = line.split(delimiter)[col] 
            list_col.append(col_word)
            
    return list_col

def save_list_to_file(list, filename, delimiter=" "):
    with open(filename, "w") as f:
        f.write(delimiter.join(list))
        
    return True

In [155]:
test_chap2_12()

['高知県', '埼玉県', '岐阜県', '山形県', '山梨県', '和歌山県', '静岡県', '山梨県', '埼玉県', '群馬県', '群馬県', '愛知県', '千葉県', '静岡県', '愛媛県', '山形県', '岐阜県', '群馬県', '千葉県', '埼玉県', '大阪府', '山梨県', '山形県', '愛知県']
['江川崎', '熊谷', '多治見', '山形', '甲府', 'かつらぎ', '天竜', '勝沼', '越谷', '館林', '上里見', '愛西', '牛久', '佐久間', '宇和島', '酒田', '美濃', '前橋', '茂原', '鳩山', '豊中', '大月', '鶴岡', '名古屋']


関数書いているから行数長くなっているけど, 実質8行ぐらいで書ける内容

savefileのassertは手抜きした

## 13. col1.txtとcol2.txtをマージ
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [165]:
def test_chap2_12():
    filename_col1_and_col2 = "col1_and_col2.txt"
    filename_col1 = "col1.txt"
    filename_col2 = "col2.txt"
    
    list_col1 = get_col_list(filename_col1, col=0, delimiter="")
    list_col2 = get_col_list(filename_col2, col=0, delimiter="")
    
    list_col1_and_col2 = extend_lists(list_col1, list_col2)
    
    paste = !paste {filename_col1} {filename_col2}
    
    assert list_col1_and_col2  == paste
    
    assert save_list_to_file(list_col1_and_col, filename_col1_and_col2, delimiter="\n") == True
    
    print(list_col1_and_col)

## 14. 先頭からN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．


## 15. 末尾のN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

## 16. ファイルをN分割する
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

## 17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはsort, uniqコマンドを用いよ．


## 18. 各行を3コラム目の数値の降順にソート
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

## 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [1]:
def test_chap2_10():
    a = !ls
    print(a)
